New Version for RUN 3 DATA/MC 

In [1]:
import ROOT
import os
from utils.samples import *
from utils.variables import *
import json
import numpy as np
import math
from dask.distributed import Client
from datetime import datetime
ROOT.RDF.Experimental.Distributed.open_files_locally = False

os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/tmp/x509up_u0"
print(os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR"))

Welcome to JupyROOT 6.27/01
/tmp/x509up_u0 /cvmfs/grid.cern.ch/etc/grid-security/certificates/


Fixed parameters definition

In [2]:
cut = requirements # ---> vedi variables.py
in_dataset = [
    "DataHTA_2018", 
    # "DataHTB_2018", 
    # "DataHTC_2018",
    # "DataHTD_2018",
    # "TprimeToTZ_700_2018", 
    # "TprimeToTZ_1000_2018", 
    # "TprimeToTZ_1800_2018",
    # "QCDHT_100to200_2018", 
    # "QCDHT_200to300_2018",
    # "QCDHT_300to500_2018", 
    # "QCDHT_500to700_2018", 
    # "QCDHT_700to1000_2018",
    # "QCDHT_1000to1500_2018", 
    # "QCDHT_1500to2000_2018",
    # "QCDHT_2000toInf_2018",
    # "TT_hadr_2018", 
    # "TT_semilep_2018", 
    # "TT_Mtt700to1000_2018", 
    # "TT_Mtt1000toInf_2018",
    # "ZJetsToNuNu_HT100to200_2018", 
    # "ZJetsToNuNu_HT200to400_2018", 
    # "ZJetsToNuNu_HT400to600_2018", 
    # "ZJetsToNuNu_HT600to800_2018", 
    # "ZJetsToNuNu_HT800to1200_2018", 
    # "ZJetsToNuNu_HT1200to2500_2018", 
    # "ZJetsToNuNu_HT2500toInf_2018"
    # "WJetsHT100to200_2018", 
    # "WJetsHT200to400_2018", 
    # "WJetsHT400to600_2018", 
    # "WJetsHT600to800_2018", 
    # "WJetsHT800to1200_2018", 
    # "WJetsHT1200to2500_2018", 
    # "WJetsHT2500toInf_2018"
]

In [3]:
# invLepveto = ""#"!" # per non invertirlo basta passare stinga vuota
# met_cut = 250
# mdphi_cut = 0
# HLT_filter = "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24"

In [4]:
## params cluster

sched_port = 24745 #Dask port
nmaxpartition = 20 # to set at lower value
distributed = True#False#
if distributed:
    nfiles_max = 999
else:
    nfiles_max = 1  #######

In [5]:
#output folder ---> waiting to implement davix lib
folder = "./results/run2018_EXO22014_presel/"# "./results/run2018_benchmark_deb/" #

if not os.path.exists(folder):
    os.mkdir(folder)
repohisto = folder+"plots/"
if not os.path.exists(repohisto):
    os.mkdir(repohisto)

In [6]:
# initialization of clusters

# upload the proxyfile to the Dask workers to make them able to access data on the grid 

from distributed.diagnostics.plugin import UploadFile
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    shutil.copyfile(working_dir + '/x509up_u0', working_dir + '/../../../x509up_u0')    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR")

text_file = open("utils/postselection.h", "r")
data = text_file.read()
def my_initialization_function():
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    print("end of initialization")

# set up everything properly
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

/usr/local/lib/python3.8/dist-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| msgpack | 1.0.3  | 1.0.2     | 1.0.3   |
| toolz   | 0.11.2 | 0.11.1    | 0.11.2  |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


0
end of initialization


Regions definitions for Analysis

In [7]:
regions_def = regions # ---> vedi variables.py
# {
#     "all_regions" : ""
#     # "resolved_1fwjet": "EventTopCategory==3 && nForwardJet>0", 
#     # "mixed_1fwjet": "EventTopCategory==2 && nForwardJet>0", 
#     # "merged_1fwjet": "EventTopCategory==1 && nForwardJet>0",
#     # "resolved_0fwjet": "EventTopCategory==3 && nForwardJet==0", 
#     # "mixed_0fwjet": "EventTopCategory==2 && nForwardJet==0", 
#     # "merged_0fwjet" : "EventTopCategory==1 && nForwardJet==0",
#     # "noTopRegion" : "EventTopCategory==0"
# }
print(regions_def)

{'NoCut': '', 'HEMVeto': '((!isMC && year == 2018) && (HEMVeto || run<319077.))', 'HEMVeto_HLT_MET_filters': '((!isMC && year == 2018) && (HEMVeto || run<319077.)) && ((HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24) && MET_filter(Flag_goodVertices, Flag_globalSuperTightHalo2016Filter, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_ecalBadCalibFilter, Flag_eeBadScFilter)) ', 'HEMVeto_HLT_MET_filters_METcut': '((!isMC && year == 2018) && (HEMVeto || run<319077.)) && ((HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24) && MET_filter(Flag_goodVertices, Flag_globalSuperTightHalo2016Filter, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilt

Importing dict samples with the info for processing

In [8]:
sample_file = open("utils/dict_samples.json", "rb")
samples = json.load(sample_file)
sample_file.close()

INPUT DATASETS

In [9]:
in_dataset

['DataHTA_2018']

In [10]:
if not in_dataset[0] in sample_dict.keys():
    datasets = []
    print("Check the in_dataset string... ", sample_dict.keys())
else : 
    datasets= []
    for d in in_dataset:
        datasets.append(sample_dict[d])
        print(datasets[-1].label)

DataHTA_2018


In [11]:
samples.keys()

dict_keys(['QCD_2018', 'QCDHT_100to200_2018', 'QCDHT_200to300_2018', 'QCDHT_300to500_2018', 'QCDHT_500to700_2018', 'QCDHT_700to1000_2018', 'QCDHT_1000to1500_2018', 'QCDHT_1500to2000_2018', 'QCDHT_2000toInf_2018', 'ZJetsToNuNu_2018', 'ZJetsToNuNu_HT100to200_2018', 'ZJetsToNuNu_HT200to400_2018', 'ZJetsToNuNu_HT400to600_2018', 'ZJetsToNuNu_HT600to800_2018', 'ZJetsToNuNu_HT800to1200_2018', 'ZJetsToNuNu_HT1200to2500_2018', 'ZJetsToNuNu_HT2500toInf_2018', 'TT_2018', 'TT_hadr_2018', 'TT_semilep_2018', 'TT_Mtt1000toInf_2018', 'TT_Mtt700to1000_2018', 'WJets_2018', 'WJetsHT100to200_2018', 'WJetsHT200to400_2018', 'WJetsHT400to600_2018', 'WJetsHT600to800_2018', 'WJetsHT800to1200_2018', 'WJetsHT1200to2500_2018', 'WJetsHT2500toInf_2018', 'TprimeToTZ_700_2018', 'TprimeToTZ_1000_2018', 'TprimeToTZ_1800_2018', 'DataHT_2018', 'DataHTA_2018', 'DataHTB_2018', 'DataHTC_2018', 'DataHT_2022', 'DataHTC_2022', 'DataHTD_2022', 'DataHTE_2022', 'DataHTF_2022', 'DataHTG_2022', 'DataHTD_2018', 'DataHTH_2016', 'Data

Variables to plot

In [12]:
#Defining variables to plot

var = vars  # ---> vedi variables.py

# var.append(variable(name = "MET_pt", title= "MET p_{T} [GeV]", taglio = cut, nbins = 20, xmin = 0, xmax=1000))
# var.append(variable(name = "MET_phi", title= "MET #phi", taglio = cut, nbins = 6, xmin = -math.pi, xmax=math.pi))
# var.append(variable(name = "LeadingJet_pt", title= "Leading Jet p_{T} [GeV]", taglio = cut, nbins = 30, xmin = 50, xmax=950))
# var.append(variable(name = "nTopHighPt", title= "# Top Candidate Mix", taglio = cut, nbins = 80, xmin = -0.5, xmax=80.5))
# var.append(variable(name = "nTopLowPt", title= "# Top Candidate Resolved", taglio = cut, nbins = 50, xmin = -0.5, xmax=50.5))
# var.append(variable(name = "nJet", title= "# Jet", taglio = cut, nbins = 25, xmin = -0.5, xmax=25.5))
# var.append(variable(name = "nFatJet", title= "# FatJet", taglio = cut, nbins = 25, xmin = -0.5, xmax=25.5))
# var.append(variable(name = "MinDelta_phi", title= "min #Delta #phi", taglio = cut, nbins = 20, xmin = 0, xmax = 4))
# var.append(variable(name = "MaxEta_jet", title= "max #eta jet", taglio = cut, nbins = 24, xmin = 0, xmax = 6))

print([v._name for v in var])


['MET_pt', 'PuppiMET_pt', 'MET_phi', 'LeadingJetPt_pt', 'LeadingJetPt_eta', 'LeadingJetPt_phi', 'LeadingJetPt_mass', 'LeadingFatJetPt_pt', 'LeadingFatJetPt_eta', 'LeadingFatJetPt_phi', 'LeadingFatJetPt_mass', 'LeadingMuonPt_pt', 'LeadingMuonPt_eta', 'LeadingMuonPt_phi', 'LeadingElectronPt_pt', 'LeadingElectronPt_eta', 'LeadingElectronPt_phi', 'nTopHighPt', 'nTopLowPt', 'nJet', 'nJetBtag', 'nFatJet', 'MinDelta_phi', 'MaxEta_jet', 'HT_eventHT', 'run', 'PV_npvsGood']


In [13]:
################### utils ###################
def cut_string(cut):
    return cut.replace(" ", "").replace("&&","_").replace(">","_g_").replace(".","_").replace("==","_e_")

################### preselection ###############
def preselection(df):
    df = df.Define("GoodJet_idx", "GetGoodJet(Jet_pt, Jet_eta, Jet_jetId)")
    df = df.Define("nGoodJet", "nGoodJet(GoodJet_idx)")
    
    # if 'leptonveto' in cut:
    #     df = df.Filter(invLepveto+"LepVeto(Electron_pt, Electron_eta, Electron_cutBased, Muon_pt, Muon_eta, Muon_looseId )", "Lepton Veto")
    #     if "&& leptonveto" in cut:
    #         c_ = cut.replace("&& leptonveto","")
    #     elif "leptonveto &&" in cut:
    #         c_ = cut.replace("leptonveto &&","")
    #     elif "leptonveto" in cut:
    #         c_ = cut.replace("leptonveto","")    
    # else: 
    #     df = df
    #     c_ = cut
    df = df.Define("nTightElectron", "nTightElectron(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("TightElectron_idx", "TightElectron_idx(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("nVetoElectron", "nVetoElectron(Electron_pt, Electron_cutBased)")
    df = df.Define("nTightMuon", "nTightMuon(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("TightMuon_idx", "TightMuon_idx(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("nVetoMuon", "nVetoMuon(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("Lepton_flavour", "Lepton_flavour(nTightElectron, nTightMuon)").Define("Lep_pt", "Lepton_var(Lepton_flavour, Electron_pt, TightElectron_idx, Muon_pt, TightMuon_idx)").Define("Lep_phi", "Lepton_var(Lepton_flavour, Electron_phi, TightElectron_idx, Muon_phi, TightMuon_idx)")
    df = df.Define("MT", "sqrt(2 * Lep_pt * MET_pt * (1 - cos(Lep_phi - MET_phi)))")
    
    # df = df.Filter("MET_pt>"+ str(met_cut), "MET_pt>"+ str(met_cut))
    # df = df.Filter("MinDelta_phi>"+ str(mdphi_cut), "MinDeltaPhi>"+ str(mdphi_cut))
    
    # df = df.Filter("atLeast1jet_setparams(Jet_pt, Jet_eta, Jet_mass, Jet_jetId, 30, 4, 0, 1)", "At_least_1Ak4")
    # df = df.Filter("atLeast1fatjet_setparams(FatJet_pt, FatJet_msoftdrop, FatJet_eta, FatJet_jetId, 200, 6, 40, 1)", "at_least_1Ak8")

    df = df.Define("LeadingJetPt_idx", "GetLeadingPtJet(Jet_pt)")
    df = df.Define("LeadingJetPt_pt", "GetLeadingJetVar(LeadingJetPt_idx, Jet_pt)")
    df = df.Define("LeadingJetPt_eta", "GetLeadingJetVar(LeadingJetPt_idx, Jet_eta)")
    df = df.Define("LeadingJetPt_phi", "GetLeadingJetVar(LeadingJetPt_idx, Jet_phi)")
    df = df.Define("LeadingJetPt_mass", "GetLeadingJetVar(LeadingJetPt_idx, Jet_mass)")
    df = df.Define("LeadingFatJetPt_idx", "GetLeadingPtJet(FatJet_pt)")
    df = df.Define("LeadingFatJetPt_pt", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_pt)")
    df = df.Define("LeadingFatJetPt_eta", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_eta)")
    df = df.Define("LeadingFatJetPt_phi", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_phi)")
    df = df.Define("LeadingFatJetPt_mass", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_mass)")
    df = df.Define("LeadingMuonPt_idx", "GetLeadingPtLep(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("LeadingMuonPt_pt", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_pt)")
    df = df.Define("LeadingMuonPt_eta", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_eta)")
    df = df.Define("LeadingMuonPt_phi", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_phi)")
    df = df.Define("LeadingElectronPt_idx", "GetLeadingPtLep(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("LeadingElectronPt_pt", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_pt)")
    df = df.Define("LeadingElectronPt_eta", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_eta)")
    df = df.Define("LeadingElectronPt_phi", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_phi)")
    
    df = df.Define("nForwardJet", "nForwardJet(GoodJet_idx, Jet_eta)")
    df = df.Define("nJetBtag", "njetbtag(GoodJet_idx, Jet_btagDeepFlavB)")
        
    return df

############### trigger selection #####################
def trigger_filter(df):
    #HLT_PFHT780 || HLT_PFHT890 || HLT_Mu50 || HLT_OldMu100 || HLT_TkMu100 || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_Ele35_WPTight_Gsf
    #preso da MET_HLT_filter
    #df_trig = df.Filter("HLT_PFHT780 || HLT_PFHT890 || HLT_Mu50 || HLT_OldMu100 || HLT_TkMu100 || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_Ele35_WPTight_Gsf")
    df_trig = df.Filter(" HLT_PFMET120_PFMHT120_IDTight || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight", "trigger")
    # controllare la lista di trigger 
    return df_trig

############### top selection ########################
def select_top(df):
    df_goodtopMer = df.Define("GoodTopMer_idx", "select_TopMer(FatJet_deepTag_TvsQCD, FatJet_pt, FatJet_eta, FatJet_phi)")
    # ritorna gli indici dei FatJet che superano la trs del Top Merged (no overlap)
    df_goodtopMix = df_goodtopMer.Define("GoodTopMix_idx", "select_TopMix(TopHighPt_score2, TopHighPt_pt, TopHighPt_eta, TopHighPt_phi)")
    # ritorna gli indici dei FatJet che superano la trs del Top Merged (no overlap)
    df_goodtopRes = df_goodtopMix.Define("GoodTopRes_idx", "select_TopRes(TopLowPt_scoreDNN, TopLowPt_pt, TopLowPt_eta, TopLowPt_phi)")
    # ritorna gli indici dei Fatche superano la trs del Top Merged (no overlap)
    df_topcategory = df_goodtopRes.Define("EventTopCategory", "select_TopCategory(GoodTopMer_idx, GoodTopMix_idx, GoodTopRes_idx)")
    # return:  0- no top sel, 1- top merged, 2- top mix, 3- top resolved
    df_topselected = df_topcategory.Define("Top_idx",
                                           "select_bestTop(EventTopCategory, GoodTopMer_idx, GoodTopMix_idx, GoodTopRes_idx, FatJet_deepTag_TvsQCD, TopHighPt_score2, TopLowPt_scoreDNN)")
    # return best top idx wrt category --> the idx is referred to the list of candidates fixed by the EventTopCategory
    df_topvariables = df_topselected.Define("Top_pt", "select_TopVar(EventTopCategory, Top_idx, FatJet_pt, TopHighPt_pt, TopLowPt_pt)")\
                        .Define("Top_eta", "select_TopVar(EventTopCategory, Top_idx, FatJet_eta, TopHighPt_eta, TopLowPt_eta)")\
                        .Define("Top_phi", "select_TopVar(EventTopCategory, Top_idx, FatJet_phi, TopHighPt_phi, TopLowPt_phi)")\
                        .Define("Top_mass", "select_TopVar(EventTopCategory, Top_idx, FatJet_mass, TopHighPt_mass, TopLowPt_mass)")\
                        .Define("Top_score", "select_TopVar(EventTopCategory, Top_idx, FatJet_deepTag_TvsQCD, TopHighPt_score2, TopLowPt_scoreDNN)")

    return df_topvariables


In [14]:
def bookhisto(df, regions_def, var, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var:
            if regions_def[reg] == "":
                h_[reg][v._name]= df.Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)
            else:
                h_[reg][v._name]= df.Filter(regions_def[reg]).Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)
    return h_

In [15]:
def savehisto(d, h, regions_def, var, s_cut):
    histo = {reg: {v._name: ROOT.TH1D(v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax) for v in var} for reg in regions_def.keys()}
    outfile = ROOT.TFile.Open(repohisto+d.label+'.root', "RECREATE")
    s = [d]
    for reg in regions_def.keys():
        for v in var:
            histo[reg][v._name] = h[reg][v._name].GetValue()      
            outfile.cd()
            histo[reg][v._name].Write()
    outfile.Close()

In [16]:
#samples['DataHTF_2022']['DataHTF_2022']['strings']

In [17]:
#samples[d.label][d.components[0]]['strings']

In [ ]:
t0 = datetime.now()
print("starting loop on datasets: ",[d.label for d in datasets])
print("Local time :", t0)
print("requirements: "+cut)

d = datasets[0]
s = datasets[0]

if 'Data' in s.label : sampleflag = 0
else: sampleflag = 1
c_ = cut
        
if nfiles_max > len(samples[d.label][s.label]['strings']): 
    nfiles = len(samples[d.label][s.label]['strings'])
    for i, string in enumerate(samples[d.label][s.label]['strings']): 
        samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
    chain = samples[d.label][s.label]['strings']
else: 
    nfiles = nfiles_max
    # for i, string in enumerate(samples[d.label][s.label]['strings']): 
    #     samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
    chain = samples[d.label][s.label]['strings'][:nfiles]
print("Initializing DataFrame for "+ s.label +" chain len = ", len(chain))
if len(chain)==1: print(chain)
if distributed ==True:
    df = RDataFrame("Events", chain, npartitions=nmaxpartition, 
                    daskclient=client, monitor_label = "main" )
else:
    df = RDataFrame("Events", chain)

df_ismc         = df.Define("isMC", "isMC("+str(sampleflag)+")")
df_year         = df_ismc.Define("year", str(s.year))
df_hemveto      = df_year.Define("HEMVeto", "hemveto(Jet_eta, Jet_phi, Electron_eta, Electron_phi)")
df_wnom         = df_hemveto.Define('w_nominal', '1')
df_presel       = preselection(df_wnom)

s_cut = cut_string(cut)
h = bookhisto(df_presel, regions_def, var, s_cut)

# if not distributed:
#     df_presel.Report().Print()

print("All histos booked !")
savehisto(d, h, regions_def, var, s_cut)
print(d.label + " finished!")
t1 = datetime.now()
print("Job finished in: ", t1-t0)

starting loop on datasets:  ['DataHTA_2018']
Local time : 2023-11-16 16:10:47.925752
requirements: 
Initializing DataFrame for DataHTA_2018 chain len =  699
All histos booked !


In [ ]:
file = ROOT.TFile.Open(repohisto+s.label+".root")
for a in file.GetListOfKeys(): print(a)

In [ ]:
regions_def.keys()

In [ ]:
region = "NoCut"
var = "MET_pt"
for reg in regions_def.keys():
    hist = file.Get(var+"_"+reg+"_")
    print(reg, hist.Integral())